## Теоретическая часть

__Задание 1.__ Вспомним прошлый вебинар, мы рассматривали **User-User** рекомендации и **Item-Item** рекомендации.
 * Чем они отличаются и чем они похожи? 
 * Если есть функция **item_item_rec**(interaction_matrix). Можно ли использовать эту функцию для **user_user_rec**?
 * В чем принципиальные отличия **item-item** рекомендаций от **ALS**?

### Ответ: 
Методы Item-based и User-based являются симметричными друг-другу с той разницей, что User-based отталкиевается от идеи, что, скажем, товар понравится пользователю, если он поравился людям со схожими интересами(предпочтениями).
$$\widehat{r_{ui}}=\overline{r_u}+\frac{\displaystyle\sum_{u\in U_i} sim(u,v)(r_{vi}-\overline{r_u})}{\displaystyle\sum_{u\in U_i} sim(u,v)}$$

В то время как Item-based показывает, что пользователю поравится товар, если ему нравились похожие товары(связанные с этим товаром неким образом).

$$\widehat{r_{ui}}=\overline{r_i}+\frac{\displaystyle\sum_{j\in I_u} sim(i,j)(r_{uj}-\overline{r_j})}{\displaystyle\sum_{j\in I_u} sim(i,j)}$$

Т.к. эти функции являются отражениями, то **item_item_rec** можно использовать для составления User-based рекоммендаций. Необходимо лишь транспонировать матрицу итераций.

item-item рекоммендации относятся к методам колобаротифной фильтрации, в то время как ALS- является методом машинного обучения. Иными словами, разница заключается в автоматизации. В случае с item-item параметры алгоритма задают и изменяют вручную, в то время как ALS сам корректирует коэффициенты при поступлении новых данных. 

Источник информации - https://www.simbirsoft.com/blog/rekomendatelnye-sistemy-v-riteyle/

__Задание 2.__ Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров 


### Ответ: 

   * quantity- к-во куленных товаров.

   * sales_value - сумма покупки

   * store_id - id магазина. В магазинах ассортимент может отличаться. Поэтому, м.б. имеет смысл использовать этот признак. 

   * week_no - номер недели. Ассортимент товаров м магазине со временем может достаточно сильно меняться. Нам интересно рекомендовать товары которые есть в магазине в данный момент. 

\* Интерессно было бы сделать комбинацию из нескольких признаков. 

Остальные признаки - касаются в основоном скидок. Нам они не интересны.

__Задание 3.__ Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)
ALS -не может работать с отрицаельными значениями.

### Ответ:

Ограничения ALS: 
1. На вход принимает только позитивные взаимодействия
2. Прогнозирует не вероятность, а некоторые числа. На выходе мы получаем некоторое число. Чем оно больше тем более вероятным является значение. Но, это не вероятность. 
3. ALS нельзя добавить внешние фичи
4. ALS не учитывает сезонность / зависимость во времени (но это можно частично поправить - см далее)


## Практическая часть


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys
from IPython.display import clear_output
from tqdm import tqdm 

tqdm.pandas()

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [19]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [20]:
data = pd.read_csv('../data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)

test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [21]:
item_features = pd.read_csv('../data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [22]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [24]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [ ]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

### Задание 1. Подбор матрицы $c_{ui}$
Попробуйте различные варианты матрицы весов (3+ вариантов). Обучите алгоритм для различных $C$. В качестве результата приведите таблицу: матрица весов - результат на train и validation.
Сделате качественные выводы.


In [26]:
def als_recomendation(values='quantity', aggfunc='count', factors=100, regularization=0.001): 
    user_item_matrix = pd.pivot_table(data_train, 
                                    index='user_id', 
                                    columns='item_id', 
                                    values= values, # Можно пробоват  и другие варианты
                                    aggfunc=aggfunc, # Проверим разные варианты агрегирующих функций.  
                                    fill_value=0
                                    )

    user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

    # переведем в формат saprse matrix
    sparse_user_item = csr_matrix(user_item_matrix).tocsr()
    # user_item_matrix.head(3)
    userids = user_item_matrix.index.values
    itemids = user_item_matrix.columns.values

    matrix_userids = np.arange(len(userids))
    matrix_itemids = np.arange(len(itemids))

    id_to_itemid = dict(zip(matrix_itemids, itemids))
    id_to_userid = dict(zip(matrix_userids, userids))

    itemid_to_id = dict(zip(itemids, matrix_itemids))
    userid_to_id = dict(zip(userids, matrix_userids))

    model = AlternatingLeastSquares(factors=factors, 
                                    regularization=regularization,
                                    iterations=15, 
                                    calculate_training_loss=True, 
                                    num_threads=4)

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
              show_progress=True)

    def get_recommendations(user, model, N=5):
        res = [id_to_itemid[rec[0]] for rec in 
                        model.recommend(userid=userid_to_id[user], 
                                        user_items=sparse_user_item,   # на вход user-item matrix
                                        N=N, 
                                        filter_already_liked_items=False, 
                                        filter_items=None, 
                                        recalculate_user=True)]
        return res
        
    result['als'] = result['user_id'].progress_apply(lambda x: get_recommendations(x, model=model, N=5))
    result_precission =  result.progress_apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
    clear_output(wait=True) # Очищаем поле вывода. 
    return result_precission

In [16]:
results_dic = { 'values':[], 'aggfunc':[], 'precission':[]} 

for values in ['quantity','sales_value','store_id']:
    for aggfunc in ['sum', 'count','mean']:
        results_dic['values'].append(values)
        results_dic['aggfunc'].append(aggfunc)
        results_dic['precission'].append(als_recomendation(values=values, aggfunc=aggfunc))
        
pd.DataFrame(results_dic)

,values,aggfunc,precission
0,quantity,sum,0.130487
1,quantity,count,0.144048
2,quantity,mean,0.155600
3,sales_value,sum,0.098543
4,sales_value,count,0.147865
5,sales_value,mean,0.126871
6,store_id,sum,0.007032
7,store_id,count,0.149473
8,store_id,mean,0.006730


### Задание 2. Оптимизация гипперпараметров
Для лучшей матрицы весов из первого задания подберите оптимальные $\lambda$ и n_factors. Подбор можно делать вручную (цикл в цикле, аналог sklearn.GridSearch, или случайно - sklearn.RandomSearch). Или Вы можете воспользоваться библиотеками для автоматического подбора гипперпараметров (любые на Ваш вкус). В качестве результата постройте графики:
1. Значение параметра - время обучения 
2. Значение параметра - качество train, качество validation  

Сделайте качественные выводы

In [29]:
%%time
results_dic = { 'factors':[], 'lambda':[], 'precission':[]} 

for factors in [10, 30, 50, 100, 150, 200, 300]:
    for regularization in [0.1, 0.01, 0.001, 0.0001]:
        results_dic['factors'].append(factors)
        results_dic['lambda'].append(regularization)
        results_dic['precission'].append(als_recomendation(values='quantity',
                                                           aggfunc='mean',
                                                           factors=factors,
                                                           regularization=regularization))
        
result_df=pd.DataFrame(results_dic)
result_df

Wall time: 2h 42min 4s


,factors,lambda,precission
0,10,0.1000,0.156605
1,10,0.0100,0.160121
2,10,0.0010,0.158011
3,10,0.0001,0.155902
4,30,0.1000,0.163837
5,30,0.0100,0.164741
6,30,0.0010,0.161025
7,30,0.0001,0.160824
8,50,0.1000,0.159719
9,50,0.0100,0.158011


Видно, что лучший результат для каждого из выбранных значений n_factors получается при $\lambda$=0.01.

Зафксируем этот параметр, чтобы посмотреть зависимость к-ва от  n_factors

In [30]:
result_df.loc[(result_df['lambda']==0.01)]

,factors,lambda,precission
1,10,0.01,0.160121
5,30,0.01,0.164741
9,50,0.01,0.158011
13,100,0.01,0.156705
17,150,0.01,0.159417
21,200,0.01,0.153993
25,300,0.01,0.140834


__Вывод:__ 

Наилучшего результата удалось добиться для небольшего к-ва факторов. (Можно уменьшить шаг и более подробно исследовать этот учаток) С одной стороны вроде бы чем больше факторов, тем точнее должен быть результат, т.к. меньше информации теряется. Однако, в данном случае - все логично. Мы имеем дело с очен сильно разреженными матрицами. Нет ничего плохого в том чтобы потерять часть незначащей информации, Если мы будем учитывать большое к-во факторов, то модель начнет предсказывать столбцы содержащие нулевые значения. И, точность предсказания упадет.